In [1]:
import glob
from os.path import basename, splitext
from inaGVAD.trs_utils import trs2df#, parsedf
from pyannote.core import Annotation, Segment, Timeline

import pandas as pd

In [2]:
def annot2df(an):
    lrec = []
    for seg, track, label in an.itertracks(yield_label=True):
        lrec.append((seg.start, seg.end, label))
    return pd.DataFrame.from_records(lrec, columns=['start', 'stop', 'label'])

def df2annotation(df):
    an = Annotation()
    uem = Timeline()
    uem.add(Segment(df.start[0], df.stop[len(df)-1]))
    vad = Annotation()

    for t in df.itertuples():
        seg = Segment(t.start, t.stop)
        vad[seg] = t.voice_activity
        if t.speaker_gender in ['male', 'female']:
            an[seg] = t.speaker_gender
        ### 
        if t.speaker_gender == 'undefgender':
            uem = uem.extrude(seg)

    return vad.support(), an.support(), uem.support()

In [3]:
for f in glob.glob('../../annotations/trs/*.trs'):
    #f = '../../annotations/trs/tv-BFT-162139.trs'
    print(f)
    bn, _ = splitext(basename(f))
    df = trs2df(f)
    df.to_csv('../../annotations/detailed_csv/%s.csv' % bn, index=False)
    
    vad, gender, uem = df2annotation(df)
    #break

    with open('../../annotations/gender_uem/%s.uem' % bn, 'wt') as fid:
        uem.write_uem(fid)
    annot2df(gender).to_csv('../../annotations/gender/%s.csv' % bn, index=False)
    annot2df(vad).to_csv('../../annotations/vad/%s.csv' % bn, index=False)
    
    

../../annotations/trs/radio-MUV-103937.trs
../../annotations/trs/radio-FBL-105429.trs
../../annotations/trs/tv-T5M-232204.trs
../../annotations/trs/tv-TF1-030233.trs
../../annotations/trs/tv-LCI-153206.trs
../../annotations/trs/radio-RMC-191036.trs
../../annotations/trs/tv-F24-045809.trs
../../annotations/trs/radio-FMU-000842.trs
../../annotations/trs/tv-BFT-183616.trs
../../annotations/trs/tv-LCI-200518.trs
../../annotations/trs/tv-LCI-213519.trs
../../annotations/trs/tv-LCI-030611.trs
../../annotations/trs/radio-FIT-193651.trs
../../annotations/trs/tv-FR4-001205.trs
../../annotations/trs/radio-FBL-051706.trs
../../annotations/trs/tv-FR3-223745.trs
../../annotations/trs/radio-RMC-030205.trs
../../annotations/trs/radio-MUV-090958.trs
../../annotations/trs/tv-FR5-102414.trs
../../annotations/trs/tv-C+_-204601.trs
../../annotations/trs/tv-F24-183254.trs
../../annotations/trs/tv-F24-055841.trs
../../annotations/trs/tv-T5M-230247.trs
../../annotations/trs/radio-FIF-164413.trs
../../annotat

/home/ddoukhan/git_repos/davidreda/env/lib/python3.8/site-packages/inaGVAD/trs_utils.py:113: UserWarning: duration 0.289 smaller than 0.300 seconds for segment <segment start=49.8 stop=50.1 label=HA>
  warnings.warn('duration %.3f smaller than %.3f seconds for segment %s' % (seg.dur, dur, seg2str(seg)))
/home/ddoukhan/git_repos/davidreda/env/lib/python3.8/site-packages/inaGVAD/trs_utils.py:113: UserWarning: duration 0.285 smaller than 0.300 seconds for segment <segment start=2.4 stop=2.7 label=BR>
  warnings.warn('duration %.3f smaller than %.3f seconds for segment %s' % (seg.dur, dur, seg2str(seg)))
/home/ddoukhan/git_repos/davidreda/env/lib/python3.8/site-packages/inaGVAD/trs_utils.py:113: UserWarning: duration 0.207 smaller than 0.300 seconds for segment <segment start=15.7 stop=15.9 label=RE>
  warnings.warn('duration %.3f smaller than %.3f seconds for segment %s' % (seg.dur, dur, seg2str(seg)))
/home/ddoukhan/git_repos/davidreda/env/lib/python3.8/site-packages/inaGVAD/trs_utils.py

../../annotations/trs/radio-FBL-011904.trs
../../annotations/trs/tv-ART-093525.trs
../../annotations/trs/tv-TF1-194413.trs
../../annotations/trs/tv-C+_-203351.trs
../../annotations/trs/tv-M6_-142753.trs
../../annotations/trs/tv-T5M-224001.trs
../../annotations/trs/radio-FBL-092729.trs
../../annotations/trs/tv-ART-174303.trs
../../annotations/trs/tv-TFX-231251.trs
../../annotations/trs/tv-T5M-072520.trs
../../annotations/trs/tv-F24-234914.trs
../../annotations/trs/tv-C+N-184644.trs
../../annotations/trs/tv-LCI-200527.trs
../../annotations/trs/tv-FR4-101704.trs
../../annotations/trs/radio-FMU-224627.trs
../../annotations/trs/tv-BFT-183240.trs
../../annotations/trs/tv-M6_-193323.trs
../../annotations/trs/tv-M6_-173630.trs
../../annotations/trs/radio-FIT-013224.trs
../../annotations/trs/tv-FR4-215612.trs
../../annotations/trs/tv-TF1-215118.trs
../../annotations/trs/radio-FBL-035452.trs
../../annotations/trs/radio-FMU-182133.trs
../../annotations/trs/radio-FIT-202337.trs
../../annotations/t

/home/ddoukhan/git_repos/davidreda/env/lib/python3.8/site-packages/inaGVAD/trs_utils.py:113: UserWarning: duration 0.247 smaller than 0.300 seconds for segment <segment start=61.0 stop=61.3 label=HA>
  warnings.warn('duration %.3f smaller than %.3f seconds for segment %s' % (seg.dur, dur, seg2str(seg)))
/home/ddoukhan/git_repos/davidreda/env/lib/python3.8/site-packages/inaGVAD/trs_utils.py:113: UserWarning: duration 0.295 smaller than 0.300 seconds for segment <segment start=55.6 stop=55.9 label=HA->
  warnings.warn('duration %.3f smaller than %.3f seconds for segment %s' % (seg.dur, dur, seg2str(seg)))
/home/ddoukhan/git_repos/davidreda/env/lib/python3.8/site-packages/inaGVAD/trs_utils.py:113: UserWarning: duration 0.248 smaller than 0.300 seconds for segment <segment start=25.4 stop=25.7 label=HA>
  warnings.warn('duration %.3f smaller than %.3f seconds for segment %s' % (seg.dur, dur, seg2str(seg)))
/home/ddoukhan/git_repos/davidreda/env/lib/python3.8/site-packages/inaGVAD/trs_utils

../../annotations/trs/tv-TFX-230837.trs
../../annotations/trs/tv-N12-132328.trs
../../annotations/trs/tv-C+N-081613.trs
../../annotations/trs/radio-FIP-222530.trs
../../annotations/trs/radio-FMU-051454.trs
../../annotations/trs/radio-FUN-013906.trs
../../annotations/trs/tv-BFT-162139.trs
../../annotations/trs/tv-FR2-161917.trs
../../annotations/trs/tv-FR4-203357.trs
../../annotations/trs/tv-FR3-124503.trs
../../annotations/trs/tv-ART-094118.trs
../../annotations/trs/radio-FIP-110726.trs
../../annotations/trs/radio-FBL-220700.trs
../../annotations/trs/radio-RTL-133218.trs
../../annotations/trs/radio-FIF-140426.trs
../../annotations/trs/radio-FIF-221229.trs
../../annotations/trs/tv-T5M-040130.trs
../../annotations/trs/radio-FBL-132313.trs
../../annotations/trs/tv-C+N-005054.trs
../../annotations/trs/radio-FCR-225430.trs
../../annotations/trs/tv-FR3-065106.trs
../../annotations/trs/tv-TF1-012319.trs
../../annotations/trs/tv-FR4-093405.trs
../../annotations/trs/radio-FCR-115618.trs
../../a

...

/home/ddoukhan/git_repos/davidreda/env/lib/python3.8/site-packages/inaGVAD/trs_utils.py:113: UserWarning: duration 0.084 smaller than 0.300 seconds for segment <segment start=0.0 stop=0.1 label=BR>
  warnings.warn('duration %.3f smaller than %.3f seconds for segment %s' % (seg.dur, dur, seg2str(seg)))
/home/ddoukhan/git_repos/davidreda/env/lib/python3.8/site-packages/inaGVAD/trs_utils.py:113: UserWarning: duration 0.280 smaller than 0.300 seconds for segment <segment start=5.3 stop=5.6 label=IA>
  warnings.warn('duration %.3f smaller than %.3f seconds for segment %s' % (seg.dur, dur, seg2str(seg)))
/home/ddoukhan/git_repos/davidreda/env/lib/python3.8/site-packages/inaGVAD/trs_utils.py:113: UserWarning: duration 0.261 smaller than 0.300 seconds for segment <segment start=6.1 stop=6.4 label=IE>
  warnings.warn('duration %.3f smaller than %.3f seconds for segment %s' % (seg.dur, dur, seg2str(seg)))
/home/ddoukhan/git_repos/davidreda/env/lib/python3.8/site-packages/inaGVAD/trs_utils.py:113

../../annotations/trs/radio-RMC-161225.trs
../../annotations/trs/tv-C+N-065324.trs
../../annotations/trs/tv-TF1-083840.trs
../../annotations/trs/radio-RTL-193856.trs
../../annotations/trs/radio-FMU-122555.trs
../../annotations/trs/tv-FR2-190223.trs
../../annotations/trs/tv-C+N-035413.trs
../../annotations/trs/radio-RMC-190319.trs
../../annotations/trs/tv-FR3-151256.trs
../../annotations/trs/tv-LCI-072215.trs
../../annotations/trs/tv-FR5-140935.trs
../../annotations/trs/radio-FMU-124550.trs
../../annotations/trs/radio-FIF-223325.trs
../../annotations/trs/tv-ART-223617.trs
../../annotations/trs/tv-T5M-221506.trs
../../annotations/trs/radio-RTL-084520.trs
../../annotations/trs/tv-M6_-165521.trs
../../annotations/trs/tv-FR4-034901.trs
../../annotations/trs/tv-T5M-211131.trs
../../annotations/trs/radio-FCR-083707.trs
../../annotations/trs/tv-FR4-172732.trs
../../annotations/trs/tv-M6_-162155.trs
../../annotations/trs/radio-FIF-145356.trs
../../annotations/trs/tv-C+N-072654.trs
../../annotat

/home/ddoukhan/git_repos/davidreda/env/lib/python3.8/site-packages/inaGVAD/trs_utils.py:113: UserWarning: duration 0.127 smaller than 0.300 seconds for segment <segment start=34.7 stop=34.8 label=FA>
  warnings.warn('duration %.3f smaller than %.3f seconds for segment %s' % (seg.dur, dur, seg2str(seg)))
/home/ddoukhan/git_repos/davidreda/env/lib/python3.8/site-packages/inaGVAD/trs_utils.py:113: UserWarning: duration 0.084 smaller than 0.300 seconds for segment <segment start=0.0 stop=0.1 label=>
  warnings.warn('duration %.3f smaller than %.3f seconds for segment %s' % (seg.dur, dur, seg2str(seg)))
/home/ddoukhan/git_repos/davidreda/env/lib/python3.8/site-packages/inaGVAD/trs_utils.py:113: UserWarning: duration 0.254 smaller than 0.300 seconds for segment <segment start=27.3 stop=27.5 label=IE>
  warnings.warn('duration %.3f smaller than %.3f seconds for segment %s' % (seg.dur, dur, seg2str(seg)))
/home/ddoukhan/git_repos/davidreda/env/lib/python3.8/site-packages/inaGVAD/trs_utils.py:1

../../annotations/trs/tv-FR2-151739.trs
../../annotations/trs/radio-FIF-111258.trs
../../annotations/trs/radio-RTL-170615.trs
../../annotations/trs/radio-FIF-121714.trs
../../annotations/trs/tv-ART-094745.trs
../../annotations/trs/tv-FR4-100835.trs
../../annotations/trs/tv-F24-182237.trs
../../annotations/trs/tv-T5M-210834.trs
../../annotations/trs/tv-C+N-071121.trs
../../annotations/trs/tv-M6_-075818.trs
../../annotations/trs/tv-T5M-073851.trs
../../annotations/trs/tv-BFT-075249.trs
../../annotations/trs/tv-TF1-183922.trs
../../annotations/trs/radio-FIF-222221.trs
../../annotations/trs/tv-LCI-100827.trs
../../annotations/trs/tv-BFT-050719.trs
../../annotations/trs/tv-BFT-170057.trs
../../annotations/trs/radio-FBL-044522.trs
../../annotations/trs/radio-FMU-135816.trs
../../annotations/trs/tv-FR4-005308.trs
../../annotations/trs/tv-C25-032841.trs
../../annotations/trs/tv-FR3-045538.trs
../../annotations/trs/tv-FR2-012411.trs
../../annotations/trs/tv-LCI-061740.trs
../../annotations/trs/

/home/ddoukhan/git_repos/davidreda/env/lib/python3.8/site-packages/inaGVAD/trs_utils.py:113: UserWarning: duration 0.206 smaller than 0.300 seconds for segment <segment start=4.0 stop=4.2 label=>
  warnings.warn('duration %.3f smaller than %.3f seconds for segment %s' % (seg.dur, dur, seg2str(seg)))
/home/ddoukhan/git_repos/davidreda/env/lib/python3.8/site-packages/inaGVAD/trs_utils.py:113: UserWarning: duration 0.254 smaller than 0.300 seconds for segment <segment start=20.8 stop=21.1 label=HA>
  warnings.warn('duration %.3f smaller than %.3f seconds for segment %s' % (seg.dur, dur, seg2str(seg)))
/home/ddoukhan/git_repos/davidreda/env/lib/python3.8/site-packages/inaGVAD/trs_utils.py:113: UserWarning: duration 0.191 smaller than 0.300 seconds for segment <segment start=25.3 stop=25.5 label=FA>
  warnings.warn('duration %.3f smaller than %.3f seconds for segment %s' % (seg.dur, dur, seg2str(seg)))
/home/ddoukhan/git_repos/davidreda/env/lib/python3.8/site-packages/inaGVAD/trs_utils.py:1

In [4]:
df

,label,start,stop,voice_activity,overlap,speaker_gender,speaker_age,speech_quality,applause,noise,hubbub,jingle,fg_music,bg_music,respiration,laughers,other,empty
0,HA,0.000,0.796,speech,False,male,adult,standard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,,0.796,2.756,nonspeech,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2,MU2,2.756,4.446,nonspeech,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN
3,HA,4.446,17.490,speech,False,male,adult,standard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,,17.490,18.745,nonspeech,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
5,BR,18.745,24.889,nonspeech,False,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,JI,24.889,26.427,nonspeech,False,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN
7,MU2,26.427,28.720,nonspeech,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN
8,FA,28.720,30.349,speech,False,female,adult,standard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,BR+MU2,30.349,31.472,nonspeech,True,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN
